<a href="https://colab.research.google.com/github/Ashi743/pytorch/blob/main/OPTUNA_ML_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 3.7 MB/s eta 0:00:00


In [ ]:
from sklearn.model_selection  import train_test_split
import optuna
from sklearn.datasets import load_diabetes
from sklearn.preprocessing import StandardScaler

import pandas as pd

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("uciml/pima-indians-diabetes-database")

print("Path to dataset files:", path)

import os
df = pd.read_csv(os.path.join(path, "diabetes.csv"))


Path to dataset files: /root/.cache/kagglehub/datasets/uciml/pima-indians-diabetes-database/versions/1


In [ ]:
import numpy as np
cols_with_missing = ['Glucose', 'BloodPressure', 'Insulin', 'BMI', 'SkinThickness']
df[cols_with_missing] = df[cols_with_missing].replace(0, np.nan)

In [ ]:
df.isnull().sum()

,0
Pregnancies,0
Glucose,0
BloodPressure,0
SkinThickness,0
Insulin,0
BMI,0
DiabetesPedigreeFunction,0
Age,0
Outcome,0


In [ ]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148.0,72.0,35.0,NaN,33.6,0.627,50,1
1,1,85.0,66.0,29.0,NaN,26.6,0.351,31,0
2,8,183.0,64.0,NaN,NaN,23.3,0.672,32,1
3,1,89.0,66.0,23.0,94.0,28.1,0.167,21,0
4,0,137.0,40.0,35.0,168.0,43.1,2.288,33,1


In [ ]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=7)
df = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

In [ ]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6.0,148.0,72.0,35.0,169.0,33.6,0.627,50.0,1.0
1,1.0,85.0,66.0,29.0,58.6,26.6,0.351,31.0,0.0
2,8.0,183.0,64.0,25.8,164.6,23.3,0.672,32.0,1.0
3,1.0,89.0,66.0,23.0,94.0,28.1,0.167,21.0,0.0
4,0.0,137.0,40.0,35.0,168.0,43.1,2.288,33.0,1.0


In [ ]:
X= df.drop('Outcome',axis=1)
y=df['Outcome']

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
scaler= StandardScaler()
X_train_scaled=scaler.fit_transform(X_train)
X_test_scaled=scaler.transform(X_test)

print(X_train.shape, X_test.shape)

(614, 8) (154, 8)


In [ ]:
!pip install xgboost
!pip install lightgbm
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 6.8 MB/s eta 0:00:00


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score


In [ ]:
import warnings
warnings.filterwarnings('ignore')
import optuna

In [ ]:
import optuna
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

def objective(trial):
    classifier_name = trial.suggest_categorical('classifier', ['RandomForest', 'SVC','LogisticRegression'])

    if classifier_name == 'RandomForest':
        n_estimators = trial.suggest_int('n_estimators', 50, 200, step=50)
        max_depth = trial.suggest_int('max_depth', 1, 10)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
        bootstrap = trial.suggest_categorical('bootstrap', [True, False])

        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth,
                                      min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf,
                                      bootstrap=bootstrap, random_state=42)

    elif classifier_name == 'SVC':
        C = trial.suggest_float('C', 0.1,10, log=True)
        kernel = trial.suggest_categorical('kernel', ['linear', 'poly', 'rbf', 'sigmoid'])
        degree = trial.suggest_int('degree', 1, 5)
        gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])

        model = SVC(C=C, kernel=kernel, degree=degree, gamma=gamma, random_state= 42)



    elif classifier_name == 'LogisticRegression':
        C_lr = trial.suggest_int('C_lr', 1,5)
        max_iter = trial.suggest_int('max_iter', 100, 1000)

        model = LogisticRegression(C=C_lr, max_iter=max_iter)

    # Perform cross-validation and return the mean accuracy score
    score = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='accuracy')
    accuracy = score.mean()

    return accuracy


In [ ]:
study= optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

[I 2025-01-19 10:52:47,843] A new study created in memory with name: no-name-cae938d7-9471-47b7-b97a-fd18c300ad24
[I 2025-01-19 10:52:47,898] Trial 0 finished with value: 0.7622284419565508 and parameters: {'classifier': 'LogisticRegression', 'C_lr': 4, 'max_iter': 725}. Best is trial 0 with value: 0.7622284419565508.
[I 2025-01-19 10:52:50,946] Trial 1 finished with value: 0.7068239370918299 and parameters: {'classifier': 'RandomForest', 'n_estimators': 150, 'max_depth': 1, 'min_samples_split': 10, 'min_samples_leaf': 10, 'bootstrap': True}. Best is trial 0 with value: 0.7622284419565508.
[I 2025-01-19 10:52:51,061] Trial 2 finished with value: 0.7622284419565508 and parameters: {'classifier': 'LogisticRegression', 'C_lr': 2, 'max_iter': 467}. Best is trial 0 with value: 0.7622284419565508.
[I 2025-01-19 10:52:56,364] Trial 3 finished with value: 0.7638811142209783 and parameters: {'classifier': 'RandomForest', 'n_estimators': 150, 'max_depth': 5, 'min_samples_split': 10, 'min_samples

In [ ]:
print(study.best_params)
print(study.best_value)

{'classifier': 'RandomForest', 'n_estimators': 50, 'max_depth': 10, 'min_samples_split': 2, 'min_samples_leaf': 10, 'bootstrap': False}
0.7737171797947487


In [ ]:
study.trials_dataframe()['params_classifier'].value_counts()

,count
params_classifier,
RandomForest,29
SVC,13
LogisticRegression,8


In [ ]:
study.trials_dataframe().groupby('params_classifier')['value'].mean()

,value
params_classifier,
LogisticRegression,0.762228
RandomForest,0.763060
SVC,0.750823


In [ ]:
from optuna.visualization import plot_optimization_history, plot_param_importances, plot_parallel_coordinate, plot_slice, plot_contour
plot_optimization_history(study)

In [ ]:
plot_slice(study).show()

In [ ]:
plot_param_importances(study).show()

In [ ]:
# Filter out the 'classifier' parameter from the best trial parameters
rf_params = {k: v for k, v in study.best_trial.params.items() if k != 'classifier'}

# Now create the RandomForestClassifier with the filtered parameters
best_model = RandomForestClassifier(**rf_params, random_state=42)
best_model.fit(X_train_scaled, y_train)
y_pred = best_model.predict(X_test_scaled)
accuracy_score(y_test, y_pred)

0.7402597402597403

In [ ]:
rf_params

{'n_estimators': 50,
 'max_depth': 10,
 'min_samples_split': 2,
 'min_samples_leaf': 10,
 'bootstrap': False}

### xgboost

In [ ]:
! pip install optuna-integration[xgboost]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 6.0 MB/s eta 0:00:00


In [ ]:
import optuna
import xgboost as xgb
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split



def objective(trial):
    params = {
        'verbose': 0,
        'objective': 'binary:logistic',  # Binary classification objective
        'eval_metric': 'logloss',  # Logarithmic loss metric for binary classification

        'booster': trial.suggest_categorical('booster', ['gbtree', 'gblinear', 'dart']),
        'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True),
        'max_depth': trial.suggest_int('max_depth', 1, 9),
        'eta': trial.suggest_float('eta', 0.01, 0.3, log=True),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'grow_policy': trial.suggest_categorical('grow_policy', ['depthwise', 'lossguide']),
        'n_estimators': 300,  # Fixed number of estimators
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.2, 1.0),
    }

    # Prepare DMatrix for XGBoost
    d_train = xgb.DMatrix(X_train_scaled, label=y_train)
    d_test = xgb.DMatrix(X_test_scaled, label=y_test)

    # Pruning callback to stop early if performance doesn't improve
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, 'test-logloss')

    # Train the model using the parameters
    best = xgb.train(
        params,
        d_train,
        evals=[(d_train, 'train'), (d_test, 'test')],
        verbose_eval=False,
        callbacks=[pruning_callback]
    )

    # Make predictions on the test set
    preds = best.predict(d_test)

    # For binary classification, we can convert probabilities to class labels
    best_pred = [int(np.round(line)) for line in preds]  # 0 or 1 prediction

    # Calculate accuracy
    accuracy = accuracy_score(y_test, best_pred)

    return accuracy


In [ ]:
# Set up the study
study = optuna.create_study(direction='maximize', pruner=optuna.pruners.SuccessiveHalvingPruner())

# Run the optimization
study.optimize(objective, n_trials=50)


[I 2025-01-19 12:05:07,526] A new study created in memory with name: no-name-967b96b9-e4b7-487d-9fe3-1053ad3b8125
[W 2025-01-19 12:05:08,138] Trial 0 failed with parameters: {'booster': 'dart', 'lambda': 0.0007208884826326686, 'alpha': 5.361185565675372e-06, 'max_depth': 6, 'eta': 0.01872130650707915, 'gamma': 5.518003065120629e-06, 'grow_policy': 'lossguide', 'colsample_bytree': 0.5672421106085382} because of the following error: NameError("name 'dtrain' is not defined").
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "<ipython-input-98-b856d7acab4e>", line 51, in objective
    return accuracy, dtrain
                     ^^^^^^
NameError: name 'dtrain' is not defined
[W 2025-01-19 12:05:08,150] Trial 0 failed with value None.


NameError: name 'dtrain' is not defined

In [ ]:
# Print the best parameters and the best accuracy
print('Best trial:', study.best_trial)
print('Best params:', study.best_trial.params)
print('Best accuracy:', study.best_value)

Best trial: FrozenTrial(number=1, state=1, values=[0.7077922077922078], datetime_start=datetime.datetime(2025, 1, 19, 11, 39, 5, 886132), datetime_complete=datetime.datetime(2025, 1, 19, 11, 39, 5, 904033), params={'booster': 'dart', 'lambda': 0.10710274525796547, 'alpha': 1.2900244813923047e-07, 'max_depth': 1, 'eta': 0.1250168727035052, 'gamma': 2.626675548795678e-06, 'grow_policy': 'depthwise', 'colsample_bytree': 0.7754994448322592}, user_attrs={}, system_attrs={'completed_rung_0': 0.6121691374422668, 'completed_rung_1': 0.5835547886498562}, intermediate_values={0: 0.6292539699511095, 1: 0.6121691374422668, 2: 0.602408251205048, 3: 0.5893051103725062, 4: 0.5835547886498562, 5: 0.5774631649255753, 6: 0.5715717766192052, 7: 0.566333456956721, 8: 0.5631334206887654, 9: 0.5597103306805933}, distributions={'booster': CategoricalDistribution(choices=('gbtree', 'gblinear', 'dart')), 'lambda': FloatDistribution(high=1.0, log=True, low=1e-08, step=None), 'alpha': FloatDistribution(high=1.0,

In [ ]:
from optuna.visualization import plot_intermediate_values
plot_intermediate_values(study).show()

In [ ]:
# Example feature matrix (X) and labels (y)
X = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])  # 3 samples, 3 features
y = np.array([0, 1, 0])  # 3 labels

# Convert to DMatrix
dtrain = xgb.DMatrix(X, label=y)

In [ ]:
features = dtrain.get_data()
print(features)

  (0, 0)	1.0
  (0, 1)	2.0
  (0, 2)	3.0
  (1, 0)	4.0
  (1, 1)	5.0
  (1, 2)	6.0
  (2, 0)	7.0
  (2, 1)	8.0
  (2, 2)	9.0
